In [16]:
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd

In [4]:
test_edges = pd.read_parquet('data/test_edges.parquet')
test_features = pd.read_parquet('data/test_features.parquet')
test_labels = pd.read_parquet('data/test_labels.parquet')

In [7]:
class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
    
    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)

In [12]:
edge_index = test_edges.loc[:, ["user_id", "product_id"]].to_numpy().T

In [13]:
data = BipartiteData(edge_index)

In [21]:
batch.edge_index

tensor([[0, 0, 1, 1, 2, 2, 3, 3],
        [0, 1, 1, 2, 3, 4, 4, 5]])